# Figure 4

The following script was used to generate the graphs of Figure 4 of the article "Behavioral Diversity Generation in Autonomous Exploration Through Reuse of Past Experience" by Fabien C. Y. Benureau and Pierre-Yves Oudeyer.

The [full code is available](http://fabien.benureau.com/code/frontiers2016.html) and is distributed under the [Open Science License](http://fabien.benureau.com/openscience.html). For any questions, remarks or difficulties running this code, contact [fabien.benureau@gmail.com](mailto:fabien.benureau@gmail.com).

In [ ]:
import random

import explorers
import environments
from environments import tools

import dotdot
import envs
import exs
import factored
import graphs

In [ ]:
DIM  = 20    # number of joint in the arm
RES  = 20    # number of row and column in the reuse grid
MB   = 50    # number of motor babbling/reuse steps
N    = 5000  # total number of steps

In [ ]:
random.seed(0) # seed used in the paper. Do experiment with other values.

## Source Task

### Instanciating the environment

In [ ]:
src_env_name, src_env_cfg = envs.kin(dim=DIM, limit=150)

src_env = environments.Environment.create(src_env_cfg)

### Instanciating the explorer

In [ ]:
src_ex_cfg = exs.catalog['rmb50.rgb.p0.05']._deepcopy()
src_ex_cfg.eras = (MB, None)
src_ex_cfg.m_channels = src_env.m_channels
src_ex_cfg.s_channels = src_env.s_channels

src_ex = explorers.Explorer.create(src_ex_cfg)

### Running the Exploration

In [ ]:
src_explorations, src_s_vectors, src_s_goals = factored.run_exploration(src_env, src_ex, N)

### Graph of the source exploration distribution of effects

In [ ]:
import graphs
graphs.output_notebook()

In [ ]:
fig = graphs.spread(src_env.s_channels, s_vectors=src_s_vectors,
                    e_radius=1.5, e_alpha=0.5, title='first arm - {} steps'.format(N))

graphs.show(fig)

### Creating reuse dataset

In [ ]:
src_dataset = {'m_channels': src_env.m_channels,
               's_channels': src_env.s_channels,
               'explorations': src_explorations}

## Target Task

In [ ]:
random.seed(3)

### Instanciating the target environment

In [ ]:
tgt_env_name, tgt_env_cfg = envs.kin(dim=DIM, limit=150, polar=True)
tgt_env_cfg.lengths = [0.9**i for i in range(tgt_env_cfg.dim)]
tgt_env_cfg.lengths = [s/sum(tgt_env_cfg.lengths) for s in tgt_env_cfg.lengths]
tgt_env = environments.Environment.create(tgt_env_cfg)

### Instanciating the reuse explorer

In [ ]:
tgt_ex_cfg                = exs.catalog['random.goal']._deepcopy()
tgt_ex_cfg.eras           = (MB, None)
tgt_ex_cfg.weights        = ((0.0, 0.0, 1.0), (0.0, 1.0, 0.0)) # mb, gb, and reuse probabilities of usage

tgt_ex_cfg.ex_2           = explorers.ReuseExplorer.defcfg._deepcopy()
tgt_ex_cfg.ex_2.reuse.res = RES

tgt_ex_cfg.m_channels = tgt_env.m_channels
tgt_ex_cfg.s_channels = tgt_env.s_channels

tgt_ex = explorers.Explorer.create(tgt_ex_cfg, datasets=[src_dataset])

### Running the exploration

In [ ]:
tgt_explorations, tgt_s_vectors, tgt_s_goals = factored.run_exploration(tgt_env, tgt_ex, N)

### Effect selection graph

In [ ]:
def reused_commands(reuse_cfg, tgt_exploration):
    uid = reuse_cfg.uuid
    expls = []
    for expl in tgt_explorations:
        if uid in expl[0]['uuids']:
            expls.append(expl)
    return expls
        
reused = reused_commands(tgt_ex.cfg.ex_2, tgt_explorations)

reused_src_s_vectors = [tools.to_vector(src_env.execute(r[0]['m_signal'])['s_signal'], src_env.s_channels) 
                        for r in reused]
reused_tgt_s_vectors = [tools.to_vector(tgt_env.execute(r[0]['m_signal'])['s_signal'], tgt_env.s_channels) 
                        for r in reused]

In [ ]:
# reuse graphs
selection_fig = graphs.spread(src_env.s_channels, s_vectors=src_s_vectors,
                              e_radius=1.5, e_alpha=0.5, title='first arm - {} steps'.format(N))
graphs.spread(src_env.s_channels, s_vectors=reused_src_s_vectors, fig=selection_fig,
              e_color='#FF030D', e_radius=2.0, e_alpha=1.0, title='first arm - {} steps'.format(N))
graphs.display_grid(selection_fig, x_range=[-1.0, 1.0], y_range=[-1.0, 1.0], 
                    div_x=RES, div_y=RES, color='#666666', alpha=0.5)

graphs.show(selection_fig)

### Illustrating reused postures on both arms

In [ ]:
src_fig = graphs.posture_explorations(src_env, reused, alpha=0.5, radius_factor=0.75, 
                                      title='{} reused commands - source exploration'.format(len(reused)))
tgt_fig = graphs.posture_explorations(tgt_env, reused, alpha=0.5, radius_factor=0.75,
                                      title='{} reused commands - target exploration'.format(len(reused)))

graphs.show([[src_fig, tgt_fig]])

### Reuse exploration graph

In [ ]:
figs = [[]]
for t in [400, N]:
    alpha = 1.0 if t != N else 0.5
    fig = graphs.spread(tgt_env.s_channels, s_vectors=tgt_s_vectors[:t],
                        e_radius=1.5, e_alpha=alpha, title='first arm - {} steps'.format(t))

    graphs.spread(tgt_env.s_channels, s_vectors=tgt_s_vectors[:MB], fig=fig,
              e_color='#FF030D', e_radius=2.0, e_alpha=1.0, title='first arm - {} steps'.format(t))

    figs[0].append(fig)

graphs.show(figs)

## No Reuse Exploration

In [ ]:
random.seed(1)

### Instanciating the explorer

In [ ]:
nor_ex_cfg = exs.catalog['random.goal']._deepcopy()
nor_ex_cfg.eras = (MB, None)
nor_ex_cfg.m_channels = tgt_env.m_channels
nor_ex_cfg.s_channels = tgt_env.s_channels

nor_ex = explorers.Explorer.create(nor_ex_cfg)

### Running the exploration

In [ ]:
nor_explorations, nor_s_vectors, nor_s_goals = factored.run_exploration(tgt_env, nor_ex, N)

### Postures generated by random motor babbling

In [ ]:
# no reuse graphs
nor_fig = graphs.posture_explorations(tgt_env, nor_explorations[:50], alpha=0.50, radius_factor=0.75, 
                                      plot_width=490, plot_height=490, 
                                      title='{} random motor commands'.format(len(reused)))

graphs.show(nor_fig)

### Random motor babbling exploration graphs

In [ ]:
figs = [[]]
for t in [400, N]:
    alpha = 1.0 if t != N else 0.5
    fig = graphs.spread(tgt_env.s_channels, s_vectors=nor_s_vectors[:t],
                        e_radius=1.5, e_alpha=alpha, title='target arm - {} steps'.format(t))
    
    graphs.spread(tgt_env.s_channels, s_vectors=nor_s_vectors[:MB], fig=fig,
              e_color='#FF030D', e_radius=2.0, e_alpha=1.0)

    figs[0].append(fig)
    
graphs.show(figs)

## Provenance Data

In [ ]:
import provenance
prov_data = provenance.planar_arms()
print(prov_data.message())